In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import pprint
from datetime import datetime, timedelta
#import tim
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import impyute as impy
from impyute.imputation.cs import mice
#from sqlalchemy import create_engine
#https://chmodi.tistory.com/114

In [2]:
import psycopg2
from sshtunnel import SSHTunnelForwarder

try:
    tunnel= SSHTunnelForwarder(
         ('210.91.223.248', 35430),
         #ssh_private_key="</path/to/private/ssh/key>",
         ### in my case, I used a password instead of a private key
         ssh_username="medicalai",
         ssh_password="medicalai731!", 
         remote_bind_address=('localhost', 35432))
         
    tunnel.start()
    print("tunnel connected")

    params = {
             'database': 'mimiciv',
             'user': 'mai_admin',
             'password': 'admin1q2w!@',
             'host': 'localhost',
             'port': tunnel.local_bind_port
             }

    conn = psycopg2.connect(**params)
    conn.autocommit = True
    curs = conn.cursor()
    print("database connected")

except Exception as e:
    print("Connection Failed due to {e}")

# Remember to close the SSH tunnel
#tunnel.close()


tunnel connected
database connected


In [3]:
curs.execute("select * from mimiciv_derived.weight_durations;")
columns_name = [desc[0] for desc in curs.description]
weight_durations = pd.DataFrame(curs.fetchall(), columns=columns_name) 

In [4]:
weight_durations

,stay_id,starttime,endtime,weight,weight_type
0,30000153,2174-09-29 10:09:00,2174-09-29 16:00:00,70.0,admit
1,30000153,2174-09-29 16:00:00,2174-10-01 05:26:10,73.0,daily
2,30000213,2162-06-21 03:38:00,2162-06-22 00:00:00,84.7,admit
3,30000213,2162-06-22 00:00:00,2162-06-22 22:52:48,73.7,daily
4,30000484,2136-01-14 15:23:32,2136-01-17 06:53:08,68.5,admit
...,...,...,...,...,...
272442,39989059,2163-10-19 12:33:55,2163-10-19 15:00:00,103.0,daily
272443,39990748,2112-12-12 17:36:53,2112-12-12 20:00:00,46.0,daily
272444,39991309,2158-06-07 08:24:12,2158-06-08 04:00:00,87.8,daily
272445,39997370,2134-03-14 15:25:06,2134-03-16 05:00:00,73.1,daily


In [6]:
weight_durations.to_csv('../outputs/weight_durations.csv', index=False, encoding='utf-8-sig')
weight_durations = pd.read_csv('../outputs/weight_durations.csv', parse_dates=['starttime','endtime'])

In [7]:
weight_durations

,stay_id,starttime,endtime,weight,weight_type
0,30000153,2174-09-29 10:09:00,2174-09-29 16:00:00,70.0,admit
1,30000153,2174-09-29 16:00:00,2174-10-01 05:26:10,73.0,daily
2,30000213,2162-06-21 03:38:00,2162-06-22 00:00:00,84.7,admit
3,30000213,2162-06-22 00:00:00,2162-06-22 22:52:48,73.7,daily
4,30000484,2136-01-14 15:23:32,2136-01-17 06:53:08,68.5,admit
...,...,...,...,...,...
272442,39989059,2163-10-19 12:33:55,2163-10-19 15:00:00,103.0,daily
272443,39990748,2112-12-12 17:36:53,2112-12-12 20:00:00,46.0,daily
272444,39991309,2158-06-07 08:24:12,2158-06-08 04:00:00,87.8,daily
272445,39997370,2134-03-14 15:25:06,2134-03-16 05:00:00,73.1,daily


In [8]:
subjectlist1 = pd.read_csv('./data/subjectlist1.csv', parse_dates=['admittime','intubationtime','dischtime', 'deathtime','extubationtime','reintubation_eventtime'])
sub1list = subjectlist1 >> select('subject_id','hadm_id','stay_id')

In [10]:
sub1_weight_durations = sub1list >> left_join(weight_durations, by=['stay_id'])

In [11]:
sub1_weight_durations.to_csv('../outputs/sub1_weight_durations.csv', index=False, encoding='utf-8-sig')

In [12]:
sub1_weight_durations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120287 entries, 0 to 120286
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   subject_id   120287 non-null  int64         
 1   hadm_id      120287 non-null  int64         
 2   stay_id      120287 non-null  int64         
 3   starttime    120103 non-null  datetime64[ns]
 4   endtime      120103 non-null  datetime64[ns]
 5   weight       120103 non-null  float64       
 6   weight_type  120103 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(3), object(1)
memory usage: 6.4+ MB
